## Testing HCF Model

#### Obtain refractive index

In [ ]:
import os
import sys

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
add_to_path = os.path.join(parent_directory, "Modules/Refractive_Indices")
os.listdir(parent_directory)
print(add_to_path)
sys.path.append(add_to_path)
import RefractiveIndexClass
import numpy as np
import matplotlib.pyplot as plt

wavelengths = np.linspace(400, 1600, 1000)
index = RefractiveIndexClass.RefractiveIndex.HCF(wavelengths=wavelengths, part="R")
plt.plot(wavelengths, index)

#### Obtain the GVD

Recall the GVD is: $$\frac{d^2 \beta}{d \omega^2}$$
We will give arguments wavelengths in nm and refractive index. We will have to perform the CDA to obtain the second derivative.

In [ ]:
def CDA2(func_vals, step_size):
    '''
    Performs the second order derivative using centered difference approximation. FDA and BDA (Ord(h^2)) obtained at https://www.dam.brown.edu/people/alcyew/handouts/numdiff.pdf
    
    ! Step size must be the same as the grid step. !
    '''
    second_derivative = []
    last_point = len(func_vals) - 1
    second_derivative.append((1 / (step_size**2)) * (2 * func_vals[0] - 5 * func_vals[1] + 4 * func_vals[2] - func_vals[3]))
    # second_derivative.append((1 / (step_size**2)) * (func_vals[2] - 2 * func_vals[1] + func_vals[0])) # If the FDA ever fails use these.
    for i in range(1, last_point):
        second_derivative.append((1 / (step_size**2)) * (func_vals[i + 1] + func_vals[i - 1] - 2 * func_vals[i]))
    # second_derivative.append((1 / (step_size**2)) * (func_vals[last_point] - 2 * func_vals[last_point - 1] + func_vals[last_point - 2])) # If the BDA ever fails use these.
    second_derivative.append((1 / (step_size**2)) * (2 * func_vals[last_point] - 5 * func_vals[last_point - 1] + 4 * func_vals[last_point - 2] - func_vals[last_point - 3]))
    return second_derivative

def CDA1(func_vals, step_size):
    '''
    Performs the second order derivative using centered difference approximation. FDA and BDA (Ord(h^2)) obtained at https://www.dam.brown.edu/people/alcyew/handouts/numdiff.pdf

    ! Step size must be the same as the grid step. !
    '''
    first_derivative = []
    last_point = len(func_vals) - 1
    first_derivative.append((1 / (2 * step_size)) * (-3 * func_vals[0] + 4 * func_vals[1] - func_vals[2] - func_vals[3]))
    for i in range(1, last_point):
        first_derivative.append((1 / (2 * step_size)) * (func_vals[i + 1] - func_vals[i - 1]))
    # first_derivative.append((1 / (step_size)) * (func_vals[last_point] - func_vals[last_point - 1]))
    first_derivative.append((1 / (2 * step_size)) * (3 * func_vals[last_point] - 4 * func_vals[last_point - 1] + func_vals[last_point - 2]))
    return first_derivative

In [ ]:
x = np.linspace(1, 10, 1000)
def y(x):
    return 2*(x-1)**5
y(x)[1000 - 1]


ypp = CDA1(y(x), step_size=x[1] - x[0])
print(ypp)
import matplotlib.pyplot as plt 
len(ypp)
plt.plot(x, ypp)
plt.plot(x, 2*5*(x-1)**4, '--')

In [ ]:
def beta_lambda(refractive_index, wavelengths):
    beta = []
    for i in range(len(refractive_index)):
        beta.append(2 * np.pi * refractive_index[i] / wavelengths[i])
    return beta

def GVD_lambda(beta, wavelengths, output_ps_nm_km = True):
    '''
    GVD which is expressed as beta_2 * (-2 pi c / lambda**2). Sometimes denoted D.

    Parameters
    -------
    beta ([float]]): Array of beta values in nm^-1
    wavelengths ([float]): Array of wavelengths corresponding to the beta array in nm
    output_ps_nm_km (bool): Output can be given in expected units from input (s / nm*nm) [False] or in conventional (ps / nm*km) [True - Default].

    Returns
    -------
    GVD as an array.
    '''
    c0 = 3e17                                                           # Speed of light in vacuum in nm / s
    first_derivative = CDA1(beta, wavelengths[1] - wavelengths[0])
    second_derivative = CDA2(beta, wavelengths[1] - wavelengths[0])
    GVD = []
    for i in range(len(beta)):
        GVD.append(-1 * ((2 * np.pi * c0) / (wavelengths[i]**2)) * ( ( (wavelengths[i]**3) / (2 * np.pi**2 * c0**2) ) * first_derivative[i] + ( (wavelengths[i]**4) / ((2 * np.pi * c0)**2) ) * second_derivative[i] ) )
    if output_ps_nm_km:
        GVD = np.array(GVD) * 1e24                                      # Converts from s / nm*nm to ps / nm*km (conventional).
    return GVD

def Vg_lambda(beta, wavelengths):
    first_derivative = CDA1(beta, wavelengths[1] - wavelengths[0])
    Vg = []
    for i in range(len(beta)):
        Vg.append( - (2 * np.pi * 3e17 / (wavelengths[i]**2)) * (1 / (first_derivative[i])))
    return Vg
n_fs = RefractiveIndexClass.RefractiveIndex.n_fs
wavelengths = np.linspace(1000, 1600, 1000)
beta = beta_lambda(n_fs(wavelengths), wavelengths)
GVD = GVD_lambda(beta, wavelengths)
len(GVD)
plt.plot(wavelengths[1: len(wavelengths) - 1], GVD[1: len(wavelengths) - 1])


In [ ]:
wavelengths = np.linspace(400, 1600, 10000)
index = RefractiveIndexClass.RefractiveIndex.HCF(wavelengths=wavelengths, R=80e-6,part="R")
plt.plot(wavelengths, (index - 1))
plt.ylim([-5e-4, 0])
plt.show()
wavelengths = wavelengths
# plt.plot(wavelengths, index)
beta_HCF = beta_lambda(index, wavelengths)
GVD_HCF = GVD_lambda(beta_HCF, wavelengths)
plt.plot(wavelengths, np.array(GVD_HCF))
plt.ylim([-10, 10])
plt.xlim(400, 1400)
plt.show()
plt.plot(wavelengths, np.array(Vg_lambda(beta_HCF, wavelengths)) / 3e17)
# plt.xlim(0.4e-6, 1.4e-6)
plt.ylim([0.9970, 1])

## <u> Gas modelling </u>

In [ ]:
gas = lambda x: RefractiveIndexClass.RefractiveIndex.Gas(wavelengths = x, pressure = 0.01, temperature = 293, gas_name="Argon")
plt.plot(wavelengths, gas(wavelengths))
plt.title("Gas index")
plt.xlabel("wavelengths")
plt.show()
# Gas must have lower index than wall.
index = RefractiveIndexClass.RefractiveIndex.HCF(wavelengths=wavelengths, R=80e-6,part="R", n_gas= gas)
print(index)
plt.plot(wavelengths, (index - 1))
plt.ylim([90.55e-3, 90.75e-3])
# plt.show()

In [ ]:
x = np.linspace(0, 2 * np.pi, 1000)
y = CDA2(np.sin(x), x[1] - x[0])
plt.plot(x, np.cos(x))
plt.plot(x, y)
plt.plot(x, -np.sin(x), '--')